In [1]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import os
import json
from pandas.io.json import json_normalize
from urllib.request import urlopen

The following two boxes compare two tables returned when making indexed call to the SiteAllocationAmountsDigest API function specifying start index and record lenth:

1. One API call with Start index 0 and record length 10
2. Two API calls with start indexes 0 and 5 and record length 5 for each and concatenate the results.

The reason for this test was I observed that the two calls were not returning similar tables, and wanted to figure out what is happening.

In [2]:
# #1 Make one call to siteallocationsDigest to access the first 10 rows of data 

url='https://wade-api-qa.azure-api.net/v1/SiteAllocationAmountsDigest?OrganizationUUID=WSDE\
&StartIndex=0&RecordCount=10'    

response =  urlopen(url)
dataread = response.read().decode("utf-8")
dataread
data = json.loads(dataread)
data
df10 = json_normalize(data, 'Sites', 
                      ['AllocationPriorityDate', 'AllocationAmount','AllocationMaximum' ]) 
df10

,Latitude,Longitude,SiteUUID,AllocationPriorityDate,AllocationAmount,AllocationMaximum
0,47.060906,-120.358700,WA_100001,2011-01-26T00:00:00,2,356.00
1,48.978377,-119.066474,WA_100002,1974-06-30T00:00:00,10,2.00
2,48.973750,-119.038081,WA_100003,1956-05-01T00:00:00,None,12.00
3,48.980615,-119.048117,WA_100005,1974-06-30T00:00:00,10,2.00
4,48.972173,-118.980836,WA_100012,1889-11-11T00:00:00,0.004,1.25
5,48.966752,-118.985362,WA_100013,1889-11-11T00:00:00,0.004,0.66
6,48.965844,-119.007295,WA_100014,1889-11-11T00:00:00,0.01,0.50
7,48.964146,-119.014000,WA_100017,1889-11-11T00:00:00,0.01,0.50
8,48.970280,-119.052122,WA_100018,1889-11-11T00:00:00,0.004,0.53
9,48.969201,-119.046863,WA_100019,1939-06-01T00:00:00,0.00222801,0.93


In [4]:
# #2 Make two calls with record lenth of 5 to access the first 10 rows of data 

df10_list = []

#index 0 record length 5
url='https://wade-api-qa.azure-api.net/v1/SiteAllocationAmountsDigest?OrganizationUUID=WSDE\
&StartIndex=0&RecordCount=5'    
response =  urlopen(url)
dataread = response.read().decode("utf-8")
dataread
data = json.loads(dataread)
data
df5 = json_normalize(data, 'Sites', 
                      ['AllocationPriorityDate', 'AllocationAmount','AllocationMaximum']) 
df10_list.append(df5)

#index 5 record length 5 (5 - 9)
url='https://wade-api-qa.azure-api.net/v1/SiteAllocationAmountsDigest?OrganizationUUID=WSDE\
&StartIndex=5&RecordCount=5'    
response =  urlopen(url)
dataread = response.read().decode("utf-8")
dataread
data = json.loads(dataread)
data
df5 = json_normalize(data, 'Sites', 
                      ['AllocationPriorityDate', 'AllocationAmount','AllocationMaximum']) 
df10_list.append(df5)

df10 = pd.concat(df10_list, sort=True, ignore_index=True)
df10

,AllocationAmount,AllocationMaximum,AllocationPriorityDate,Latitude,Longitude,SiteUUID
0,2,356,2011-01-26T00:00:00,47.060906,-120.358700,WA_100001
1,10,2,1974-06-30T00:00:00,48.978377,-119.066474,WA_100002
2,None,12,1956-05-01T00:00:00,48.973750,-119.038081,WA_100003
3,10,2,1974-06-30T00:00:00,48.980615,-119.048117,WA_100005
4,0.004,1.25,1889-11-11T00:00:00,48.972173,-118.980836,WA_100012
5,0.0668403,None,1997-05-07T00:00:00,47.885198,-120.060041,WA_86311
6,0.0668403,None,1997-05-07T00:00:00,47.883498,-120.057917,WA_86312
7,3.9,None,1997-04-18T00:00:00,47.313119,-120.084649,WA_20910
8,0.03,None,1997-09-11T00:00:00,47.830231,-120.778626,WA_84867
9,0.02,None,1997-06-18T00:00:00,46.916877,-121.036292,WA_113219
